# Machine learning to predict energy consumption

Introduction text

### Import packages

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import matplotlib.pyplot as plt
%matplotlib inline

### Clean and prepare the datasets

Which data is there? 

#### Import the datasets

In [2]:
inflation_Netherlands = pd.read_excel("Inflationfile.xlsx", skiprows = range(265, 273), index_col = False)
energy_consumption = pd.read_csv("MontlyEnergy.csv", skiprows = range(265, 273), index_col = 0)
sun_hours = pd.read_csv("zonuren1.csv", index_col = False)
temperature = pd.read_csv("Dataset_temperatuur.csv", skiprows = range(265, 274), index_col = False)
wind_mode = pd.read_csv("WeatherMontlyMode.csv", skiprows = range(265, 275), index_col = False)
wind_mean = pd.read_csv("WeatherMontlyAverage.csv", skiprows = range(265, 275), index_col = False)

#### Make a common date column

In [3]:
# Inflation dataset
import locale
locale.setlocale(locale.LC_ALL, "nl_NL")

inflation_Netherlands['Perioden'] = pd.to_datetime(inflation_Netherlands['Perioden'], format='%Y %B').apply(lambda date: date.strftime(format='%Y%m'))

# Energy consumption dataset
energy_consumption['Perioden'] = pd.to_datetime(energy_consumption['Perioden'], format='%YMM%m').apply(lambda date: date.strftime(format='%Y%m'))

# Sun hours dataset
sun_hours['Datum'] = pd.to_datetime(sun_hours['Datum'], format='%Y-%m').apply(lambda date: date.strftime(format='%Y%m'))

# Temperature dataset
temperature['DateTime'] = pd.to_datetime(temperature['DateTime'], format='%Y-%m').apply(lambda date: date.strftime(format='%Y%m'))

# Wind datasets
wind_mode['YYYYMMDD'] = pd.to_datetime(wind_mode['YYYYMMDD'], format='%Y-%m').apply(lambda date: date.strftime(format='%Y%m'))
wind_mean['YYYYMMDD'] = pd.to_datetime(wind_mean['YYYYMMDD'], format='%Y-%m').apply(lambda date: date.strftime(format='%Y%m'))

#### Rename the columns

In [4]:
# Inflation dataset
inflation_Netherlands01 = inflation_Netherlands.rename(columns={'Perioden': 'Date'})

# Energy consumption dataset
energy_consumption01 = energy_consumption.rename(columns={'Perioden': 'Date'})

# Sun hours dataset
sun_hours01 = sun_hours.rename(columns={'Datum': 'Date'})

# Temperature dataset
temperature01 = temperature.rename(columns={'DateTime': 'Date'})

# Wind datasets
wind_mode01 = wind_mode.rename(columns={'YYYYMMDD': 'Date'})
wind_mean01 = wind_mean.rename(columns={'YYYYMMDD': 'Date'})


#### Drop some columns

In [5]:
# Inflation dataset
inflation_Netherlands02 = inflation_Netherlands01.drop(['Jaar', 'Maand'], axis=1)

# Energy dataset
energy_consumption02 = energy_consumption01.drop(['ID'], axis=1)

# Temperature dataset
temperature02 = temperature01.drop(['Unnamed: 0'], axis=1)

#### Merge the datasets

In [6]:
# merge all the dataframes
ml_datasetenergy = inflation_Netherlands02.merge(energy_consumption02, on='Date', how='left').merge(sun_hours01, on='Date', how='left').merge(temperature02, on='Date', how='left').merge(wind_mode01, on='Date', how='left').merge(wind_mean01, on='Date', how='left')

In [7]:
ml_datasetenergy

,Date,CPI_energie,CPI_energie-100,CPI(afgeleid)_energie,CPI_algemeen,CPI(algemeen)_afgeleid,CPI_elektriciteit,CPI(elektriciteit)_afgeleid,CPI_gas,CPI(gas)_afgeleid,CPI_energiejaarlijks,NettoVerbruikBerekend_30,zonuren_dag,TG,TN,TX,FG_x,FG_y
0,200001,52.22,-47.78,65.20,73.55,77.17,69.63,78.38,43.70,57.67,9.3,8847,1.806452,4.329032,1.729032,6.561290,4.0,4.219355
1,200002,52.29,-47.71,65.28,74.00,77.64,69.84,78.62,43.69,57.66,9.4,8235,3.682759,5.855172,2.537931,8.941379,4.6,4.748276
2,200003,52.73,-47.27,65.84,74.66,78.33,71.36,80.33,43.62,57.56,10.3,8738,2.583871,6.796774,3.361290,10.112903,1.5,3.980645
3,200004,52.74,-47.26,65.85,74.78,78.46,71.38,80.35,43.62,57.56,10.7,7912,5.116667,9.986667,5.186667,14.753333,1.9,3.553333
4,200005,52.74,-47.26,65.85,75.02,78.69,71.38,80.35,43.62,57.56,10.7,8221,6.780645,14.732258,9.274194,20.022581,3.8,3.496774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,202108,117.17,17.17,112.55,110.71,108.87,79.68,135.55,139.54,100.92,13.7,8981,5.325806,16.935484,12.735484,20.906452,3.5,2.958065
260,202109,123.29,23.29,118.37,110.79,108.94,85.71,144.98,145.74,105.53,19.4,9030,5.940000,15.930000,10.826667,21.186667,2.4,2.510000
261,202110,136.56,36.56,130.99,112.18,110.31,96.27,161.47,161.18,116.98,32.1,9410,3.945161,11.612903,7.541935,15.558065,3.6,3.335484
262,202111,163.61,63.61,156.72,113.17,111.29,121.57,201.01,189.65,138.11,57.4,9678,1.946667,7.380000,4.420000,10.056667,1.8,2.723333
